## Predicting Results of US Elections on a County Level

#### Table of Contents <a name='top'></a>

- [Load Modules and Set Notebook Properties](#modules)
- [Define Path and Load Data](#load)
- [Inspect Data](#inspect)
- [Define a State Selection Function ](#stateselect)
- [Define Scaling Function](#scale)
- [Create Different Classifier Models](#create)
- [Find the Best Model](#evaluate)
- [Evaluate and Choose Models](#evaluate)
- [Predict](#predict)
- [Prepare Submission](#submit)

[go to end](#end)

#### Load Modules and Set Notebook Properties <a name='modules'></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import sys
import os

In [2]:
from config import ALL_STATES

In [40]:
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, StandardScaler, MinMaxScaler, Normalizer

In [68]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [23]:
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
sns.set_style("darkgrid")

#### Define Path and Load Data  <a name='load'></a> 

In [24]:
OUTPUT_PATH = 'outputs'

In [25]:
df = pd.read_csv(os.path.join(OUTPUT_PATH, 'results_2020.csv'))
df_ = df[['county_fips', 'state', 'winner']]

In [26]:
df2 = pd.read_csv(os.path.join(OUTPUT_PATH, 'rate_natural_increase_population_2020.csv'))
df3 = pd.read_csv(os.path.join(OUTPUT_PATH, 'rate_international_migration_2020.csv'))
df4 = pd.read_csv(os.path.join(OUTPUT_PATH, 'rate_domestic_migration_2020.csv'))
df5 = pd.read_csv(os.path.join(OUTPUT_PATH, 'facebook_2014.csv'))
df6 = pd.read_csv(os.path.join(OUTPUT_PATH, 'social_capital.csv')) 

In [27]:
merged = df_.merge(df2, on='county_fips', how='left')\
            .merge(df3, on='county_fips', how='left')\
            .merge(df4, on='county_fips', how='left')\
            .merge(df5, on='county_fips', how='left')\
            .merge(df6, on='county_fips', how='left')

In [28]:
merged.fillna(0, inplace=True)

#### Inspect Data  <a name='inspect'></a> 

In [29]:
merged.head()

,county_fips,state,winner,rate_natural_increase_population,rate_international_migration,rate_domestic_migration,sh050m,sh100m,sh500m,Religious2014,Civic2014,Business2014,Political2014,Professional2014,Labor2014,Bowling2014,Recreational2014,Golf2014,Sports2014,assn2014,pvote2012,respn2010,nccs2014,sk2014
0,1001.0,ALABAMA,republican,1.490099,-0.287248,4.847310,0.206411,0.350840,0.291315,53.0,7.0,3.0,0.0,1.0,4.0,1.0,5.0,2.0,0.0,1.374570,0.644956,0.78,157.0,-0.631003
1,1003.0,ALABAMA,republican,-0.099753,0.362739,24.017829,-0.099281,-0.641534,-0.209279,169.0,16.0,9.0,0.0,3.0,1.0,1.0,25.0,7.0,0.0,1.156660,0.674735,0.73,718.0,-0.555396
2,1005.0,ALABAMA,republican,-2.259978,0.524638,-5.690302,0.359257,0.516236,0.666761,19.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.820436,0.665031,0.63,92.0,-0.891036
3,1007.0,ALABAMA,republican,-0.536181,0.446818,1.385134,1.582025,1.260516,0.917058,20.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.020001,0.656838,0.58,54.0,-0.906582
4,1009.0,ALABAMA,republican,-0.103809,0.103809,1.020788,1.582025,1.260516,0.917058,39.0,0.0,1.0,0.0,0.0,3.0,0.0,3.0,3.0,0.0,0.849839,0.708668,0.80,108.0,-1.013280


#### Define a State Selection Function <a name='stateselect'></a> 

In [30]:
def select_states(states=ALL_STATES):
    
    return merged[merged['state'].isin(states)]

#### Define Scaling Function <a name='scale'></a> 

In [31]:
def scale_values(X_train, X_test, scaler='standard'):
    
    scaler_dict = {'standard': StandardScaler(), 
                    'minmax': MinMaxScaler(), 
                    'normal': Normalizer()}
    if scaler is None:
        return X_train, X_test
    elif scaler not in scaler_dict.keys():
        raise ValueError("Enter a valid value for scaler! Choose between 'standard', 'minmax', 'normal' or None.")
    else:
        scl = scaler_dict[scaler]
        X_train = scl.fit_transform(X_train)
        X_test = scl.transform(X_test) 
        return X_train, X_test

#### Define Classifiers

In [32]:
knn = KNeighborsClassifier(n_neighbors=5)
svc = SVC()
log_reg =  LogisticRegression(penalty="l2", C=1e42, solver='liblinear')
nb = BernoulliNB(binarize=True)
dt = DecisionTreeClassifier(criterion = 'entropy', 
                            max_depth=3, 
                            random_state = 0)
xgb = XGBClassifier(n_estimators=100, 
                    objective='binary:logistic', 
                    use_label_encoder=False, 
                    eval_metric='error')

In [33]:
classifier_dict = {'k-Nearest Neighbors': knn,
                   'Support Vector': svc,
                   'Logistic Regression': log_reg,
                   'Naive Bayes': nb,
                   'Decision Tree': dt,
                   'XGBoost': xgb}

#### Test Classifiers without Hyperparameter Tuning

In [59]:
final = select_states(['ARIZONA', 'GEORGIA', 'PENNSYLVANIA', 'FLORIDA', 'OHIO', 'MICHIGAN', 'WISCONSIN'])
X = final.drop(['county_fips', 'state', 'winner'], axis=1)
y = [1 if x == 'republican' else 0 for x in final['winner']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=27)
X_train, X_test = scale_values(X_train, X_test, scaler='minmax')

In [60]:
print('Accuracy Scores')
print('===============')
for i in classifier_dict:
    clf = classifier_dict[i]
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    accuracy = accuracy_score(pred, y_test)
    print('{}: {:.4f}'.format(i, accuracy))

Accuracy Scores
k-Nearest Neighbors: 0.8855
Support Vector: 0.8855
Logistic Regression: 0.9036
Naive Bayes: 0.8554
Decision Tree: 0.8735
XGBoost: 0.8976


#### Tune Classifiers

In [107]:
knn2 = KNeighborsClassifier()
svc2 = SVC()
log_reg2 =  LogisticRegression()
dt2 = DecisionTreeClassifier()
xgb2 = XGBClassifier()

In [108]:
knn_params = {'n_neighbors': list(range(1,20))}
svc_params = {'C': np.logspace(-3, 2, 6), 
              'gamma': np.logspace(-3, 2, 6)}
logreg_params = {'solver': ['newton-cg', 'lbfgs', 'liblinear'],
                 'penalty': ['l2'],
                 'C': [100, 10, 1.0, 0.1, 0.01]}
dt_params = {'criterion': ['gini', 'entropy'],
             'max_depth': range(1,10),
             'min_samples_split': range(1,10),
             'min_samples_leaf': range(1,10)}
xgb_params = {"learning_rate": [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ],
              'n_estimators': [10, 20, 40, 100],
              'objective': ['binary:logistic'],
              'use_label_encoder': [False],
              'eval_metric': ['error'],
              "max_depth": [ 3, 4, 5, 6, 8, 10, 12, 15],
              "min_child_weight": [ 1, 3, 5, 7 ],
              "gamma": [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
              "colsample_bytree": [ 0.3, 0.4, 0.5 , 0.7 ] }

In [110]:
classifier_dict2 = {'k-Nearest Neighbors': [knn2, knn_params],
                    'Support Vector': [svc2, svc_params],
                    'Logistic Regression': [log_reg2, logreg_params],
                    'Decision Tree': [dt2, dt_params],
                    'XGBoost': [xgb2, xgb_params]}

In [ ]:
print('Hyperparameter Optimization')
print('===========================')
for i in classifier_dict2:
    print(i)
    clf = classifier_dict2[i][0]
    params = classifier_dict2[i][1]
    grid = GridSearchCV(clf, params, cv=3, scoring='accuracy')
    grid.fit(X_train, y_train)
    print(f'Best Parameters: {grid.best_params_}')
    print('Accuracy on Training: {:.4f}'.format(grid.best_score_))
    
    best_model = grid.best_estimator_
    pred = best_model.predict(X_test)
    accuracy = accuracy_score(pred, y_test)
    print('Accuracy on Test: {:.4f}'.format(accuracy))
    print('+++++++++++++++++++++++++++')

Hyperparameter Optimization
k-Nearest Neighbors
Best Parameters: {'n_neighbors': 9}
Accuracy on Training: 0.8780
Accuracy on Test: 0.8735
+++++++++++++++++++++++++++
Support Vector
Best Parameters: {'C': 100.0, 'gamma': 0.1}
Accuracy on Training: 0.8832
Accuracy on Test: 0.9096
+++++++++++++++++++++++++++
Logistic Regression
Best Parameters: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
Accuracy on Training: 0.8858
Accuracy on Test: 0.8916
+++++++++++++++++++++++++++
Decision Tree
Best Parameters: {'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 7, 'min_samples_split': 6}
Accuracy on Training: 0.8858
Accuracy on Test: 0.8735
+++++++++++++++++++++++++++
XGBoost
